In [1]:
import warnings # 忽略警告
warnings.simplefilter("ignore")
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from utils import get_prompt, get_bnb_config

bin d:\Users\Ray Lee\anaconda3\envs\adl3\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
model_name = "yentinglin/Taiwan-LLM-7B-v2.0-chat"

new_model = "1129"

model_path = "./1129"  # 更改為您的路徑

In [4]:
# Quantized LLMs with Low-Rank Adapters (QLoRA) parameters
################################################################################
lora_r = 4
lora_alpha = 16
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters 輕量級封裝，專門用於CUDA自定義函數，特別是8位優化器、矩陣乘法和量化
################################################################################
use_4bit = True
bnb_4bit_compute_dtype = "bfloat16" # float16 or bfloat16
bnb_4bit_quant_type = "nf4" # fp4 or nf4
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 5
logging_steps = 5

################################################################################
# Supervised finetuning (SFT) parameters
################################################################################
max_seq_length = None
packing = False
device_map = {"": 0} #{"": 0} or "auto"

In [5]:
train_dataset = load_dataset('json', data_files='G:/HW3/data/train_1.json', split="train")  # 從JSON文件中載入訓練數據集
valid_dataset = load_dataset('json', data_files='G:/HW3/data/valid_1.json', split="train")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1001.74it/s]
Generating train split: 3200 examples [00:00, 34114.77 examples/s]
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1001.03it/s]
Generating train split: 800 examples [00:00, 160447.72 examples/s]


In [6]:
train_dataset = train_dataset.map(
    lambda examples: {
        'instruction': [get_prompt(instruction) for instruction in examples['instruction']],
        'output': examples['output']
    },
    batched=True
)

Map: 100%|██████████| 3200/3200 [00:00<00:00, 84409.95 examples/s]


In [7]:
valid_dataset = valid_dataset.map(
    lambda examples: {
        'instruction': [get_prompt(instruction) for instruction in examples['instruction']],
        'output': examples['output']
    },
    batched=True
)

Map: 100%|██████████| 800/800 [00:00<00:00, 50136.62 examples/s]


In [8]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# 檢查 GPU 是否與 bfloat16 相容
if compute_dtype == torch.bfloat16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [9]:
# 載入與模型對應的分詞器
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [10]:
# Example of freeing GPU memory
torch.cuda.empty_cache()

In [12]:
# 從預訓練模型中載入自動生成模型
model = AutoModelForCausalLM.from_pretrained(
    "../Taiwan-LLM-7B-v2.0-chat",
    quantization_config=get_bnb_config(),
    torch_dtype=torch.bfloat16,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.88s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ../Taiwan-LLM-7B-v2.0-chat and are newly initialized: ['model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.la

In [13]:
# 定義 Prompt Engineering Fine-Tuning （PEFT）的相關設定
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [14]:
# 設置訓練參數
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard", #"all"
    evaluation_strategy="steps",
    eval_steps=5  # 每5步驗證
)

In [15]:
# 使用 SFTTrainer 進行監督式微調訓練
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset, # 在這裡傳入驗證集
    eval_dataset=valid_dataset,
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# 開始訓練模型
trainer.train()

  1%|          | 5/800 [00:12<26:35,  2.01s/it]  

{'loss': 1.6367, 'learning_rate': 4.166666666666667e-05, 'epoch': 0.01}


                                               
  1%|          | 5/800 [01:54<26:35,  2.01s/it]  

{'eval_loss': 1.3150025606155396, 'eval_runtime': 102.3725, 'eval_samples_per_second': 7.815, 'eval_steps_per_second': 0.977, 'epoch': 0.01}


  1%|▏         | 10/800 [02:02<2:01:03,  9.19s/it]

...

                                                   
100%|██████████| 800/800 [4:50:06<00:00, 10.00s/it]

{'eval_loss': 0.7785902619361877, 'eval_runtime': 103.4597, 'eval_samples_per_second': 7.732, 'eval_steps_per_second': 0.967, 'epoch': 1.0}


100%|██████████| 800/800 [4:50:06<00:00, 21.76s/it]

{'train_runtime': 17406.9139, 'train_samples_per_second': 0.184, 'train_steps_per_second': 0.046, 'train_loss': 0.783082422465086, 'epoch': 1.0}


TrainOutput(global_step=800, training_loss=0.783082422465086, metrics={'train_runtime': 17406.9139, 'train_samples_per_second': 0.184, 'train_steps_per_second': 0.046, 'train_loss': 0.783082422465086, 'epoch': 1.0})

In [16]:
trainer.model.save_pretrained(new_model)

In [17]:
# 以FP16重新載入模型並將其與LoRA權重合併
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# 重新載入分詞器以進行保存
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 儲存合併後的模型
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.98s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at yentinglin/Taiwan-LLM-7B-v2.0-chat and are newly initialized: ['model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', '

('./1129\\tokenizer_config.json',
 './1129\\special_tokens_map.json',
 './1129\\tokenizer.model',
 './1129\\added_tokens.json',
 './1129\\tokenizer.json')